# Segmenting and Clustering Neighborhoods in Toronto

In [3]:
# Install packages
import sys
!{sys.executable} -m pip install geocoder
!{sys.executable} -m pip install folium
!pip install BeautifulSoup4
# Import libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import geocoder # import geocoder
import requests 
from bs4 import BeautifulSoup
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library
print('Libraries imported.')

     |████████████████████████████████| 102kB 9.9MB/s eta 0:00:01
     |████████████████████████████████| 102kB 8.0MB/s ta 0:00:011
Libraries imported.


In [4]:
# Scrape Wikipedia page
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 
#print(soup.prettify()) 
print('Page Scrapped.')

Page Scrapped.


In [5]:
# Collect data
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell == 'Not assigned\n':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1

print('Data Collected.')

Data Collected.


In [6]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighbors = pd.DataFrame(columns=column_names)

neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [7]:
lat_lng_coords = None

for data in range(0, len(postalCodes)):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name,
                                   'Latitude': lat_lng_coords[0],
                                   'Longitude': lat_lng_coords[1]}, ignore_index=True)
    
neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75188,-79.33036
1,M4A,North York,Victoria Village,43.73042,-79.31282
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65514,-79.36265
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72321,-79.45141
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66449,-79.39302
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.66277,-79.52831
6,M1B,Scarborough,"Malvern, Rouge",43.81153,-79.19552
7,M3B,North York,Don Mills,43.74929,-79.36169
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.70794,-79.31160
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.65736,-79.37818


In [8]:
# Get url from Foursquare
VERSION = '20180605' # Foursquare API version

neighborhood_name = neighbors.loc[0, 'Neighborhood'] # neighborhood name
neighborhood_latitude = neighbors.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighbors.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

# Define parameters
radius = 500 # define radius
LIMIT = 100 # limit of number of venues returned by Foursquare API
CLIENT_ID = 'IN1UG4DWYYHJNTGN3FQDS4WQJSDY3EAMIVLFSFCODVPHKX04'
CLIENT_SECRET = 'OA3KX5EQDJJHYUSDXOFDJQ2V0XEYJT4EOM5IMHLVHNA1JSOG'

# Get url
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
     CLIENT_ID, 
     CLIENT_SECRET, 
     VERSION, 
     43.67635739999999, 
     -79.2930312, 
     radius, 
     LIMIT)

results = requests.get(url).json()
results

Latitude and longitude values of Parkwoods
 are 43.75188000000003, -79.33035999999998.


{'meta': {'code': 200, 'requestId': '5f291de4b746232b30243a4d'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.29394208780985}],
        'distanc

In [9]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [10]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [11]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [12]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
             CLIENT_ID, 
             CLIENT_SECRET, 
             VERSION, 
             lat, 
             lng, 
             radius, 
             LIMIT)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
toronto_venues = getNearbyVenues(names=neighborhoods,
                                 latitudes=neighbors['Latitude'],
                                 longitudes=neighbors['Longitude'])

Parkwoods

Victoria Village

Regent Park, Harbourfront

Lawrence Manor, Lawrence Heights

Queen's Park, Ontario Provincial Government

Islington Avenue, Humber Valley Village

Malvern, Rouge

Don Mills

Parkview Hill, Woodbine Gardens

Garden District, Ryerson

Glencairn

West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale

Rouge Hill, Port Union, Highland Creek

Don Mills

Woodbine Heights

St. James Town

Humewood-Cedarvale

Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood

Guildwood, Morningside, West Hill

The Beaches

Berczy Park

Caledonia-Fairbanks

Woburn

Leaside

Central Bay Street

Christie

Cedarbrae

Hillcrest Village

Bathurst Manor, Wilson Heights, Downsview North

Thorncliffe Park

Richmond, Adelaide, King

Dufferin, Dovercourt Village

Scarborough Village

Fairview, Henry Farm, Oriole

Northwood Park, York University

East Toronto, Broadview North (Old East York)

Harbourfront East, Union Station, Toronto Islands

Little Portugal, Trinit

In [14]:
# check the size of the resulting dataframe

print(toronto_venues.shape)
toronto_venues.head()

(2420, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.75188,-79.33036,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.75188,-79.33036,PetSmart,43.748639,-79.333488,Pet Store
2,Parkwoods,43.75188,-79.33036,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Parkwoods,43.75188,-79.33036,Sun Life,43.754760,-79.332783,Construction & Landscaping
4,Victoria Village,43.73042,-79.31282,Memories of Africa,43.726602,-79.312427,Grocery Store


In [15]:
# check how many venues were returned for each neighborhood

toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,8,8,8,8,8,8
"Alderwood, Long Branch",5,5,5,5,5,5
"Bathurst Manor, Wilson Heights, Downsview North",1,1,1,1,1,1
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,67,67,67,67,67,67
"Birch Cliff, Cliffside West",5,5,5,5,5,5
"Brockton, Parkdale Village, Exhibition Place",84,84,84,84,84,84
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",100,100,100,100,100,100


In [16]:
# find out how many unique categories can be curated from all the returned venues

print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 261 uniques categories.


In [17]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# examine the new dataframe size.

toronto_onehot.shape

(2420, 261)

In [19]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00
1,"Alderwood, Long Branch",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00
2,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00
3,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,...,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00
4,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00
5,Berczy Park,0.014925,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.014925,0.000000,...,0.000000,0.000000,0.000000,0.014925,0.000000,0.000000,0.000000,0.00,0.000000,0.00
6,"Birch Cliff, Cliffside West",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00
7,"Brockton, Parkdale Village, Exhibition Place",0.011905,0.011905,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.011905,0.000000,0.000000,0.000000,0.00,0.000000,0.00
8,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.020000,0.00,0.0,0.010000,0.000000,...,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.010000,0.00,0.000000,0.00
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.014706,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.014706,0.00


In [20]:
# confirm the new size

toronto_grouped.shape

(98, 261)

In [21]:
# print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt
----
              venue  freq
0              Pool  0.12
1  Sushi Restaurant  0.12
2   Badminton Court  0.12
3              Park  0.12
4      Skating Rink  0.12


----Alderwood, Long Branch
----
               venue  freq
0                Gym   0.2
1  Convenience Store   0.2
2        Pizza Place   0.2
3     Sandwich Place   0.2
4                Pub   0.2


----Bathurst Manor, Wilson Heights, Downsview North
----
                       venue  freq
0                       Park   1.0
1                Yoga Studio   0.0
2                 Nail Salon   0.0
3  Middle Eastern Restaurant   0.0
4         Miscellaneous Shop   0.0


----Bayview Village
----
                        venue  freq
0                       Trail  0.25
1                        Park  0.25
2                     Dog Run  0.25
3  Construction & Landscaping  0.25
4          Miscellaneous Shop  0.00


----Bedford Park, Lawrence Manor East
----
                venue  freq
0         Coffee Shop  0.09
1    Sushi Resta

In [22]:
# sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
# create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Agincourt,Park,Discount Store,Badminton Court
1,"Alderwood, Long Branch",Pizza Place,Pub,Gym
2,"Bathurst Manor, Wilson Heights, Downsview North",Park,Women's Store,Discount Store
3,Bayview Village,Dog Run,Construction & Landscaping,Park
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Sandwich Place


In [45]:
# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]
#print(kmeans.labels_)

[0 0 3 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 3 0 0 0 1 0 1
 0 0 0 0 0 0 0 0 0 0 1 0 0 2 4 0 1 1 1 1 0 0 0 0 0 1 0 0 0 1 2 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0]


In [54]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
#print(neighborhoods_venues_sorted['Cluster Labels'])

toronto_merged = neighbors

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# fix label bug
toronto_merged['Cluster Labels'] = int(str(toronto_merged['Cluster Labels'])[0])

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,M3A,North York,Parkwoods,43.75188,-79.33036,0,Pet Store,Construction & Landscaping,Park
1,M4A,North York,Victoria Village,43.73042,-79.31282,0,Grocery Store,Intersection,Nail Salon
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65514,-79.36265,0,Coffee Shop,Restaurant,Breakfast Spot
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72321,-79.45141,0,Clothing Store,Furniture / Home Store,Women's Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66449,-79.39302,0,Coffee Shop,Sandwich Place,Park


In [55]:
# Finally, let's visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[43.67635739999999, -79.2930312], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    #print(cluster)
    cluster = int(str(cluster)[0])
    #print(cluster)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [56]:
# CLUSTER 1

toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,North York,0,Pet Store,Construction & Landscaping,Park
1,North York,0,Grocery Store,Intersection,Nail Salon
2,Downtown Toronto,0,Coffee Shop,Restaurant,Breakfast Spot
3,North York,0,Clothing Store,Furniture / Home Store,Women's Store
4,Downtown Toronto,0,Coffee Shop,Sandwich Place,Park
5,Etobicoke,0,Pharmacy,Shopping Mall,Café
6,Scarborough,0,Home Service,Furniture / Home Store,Distribution Center
7,North York,0,Coffee Shop,Gas Station,Beer Store
8,East York,0,Pizza Place,Pharmacy,Athletics & Sports
9,Downtown Toronto,0,Coffee Shop,Clothing Store,Hotel
